# For Savannah

In [10]:
# import modules
import numpy as np
import astropy.units as u

# import plotting module
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# import previous code
from ReadFile import Read
from CenterOfMass import CenterOfMass  # try COM instead of COM2
#from ParticleProperties import ParticleProperties

# for contours
import scipy.optimize as so

In [59]:
def AngularMomentum(filename, ptype, r_max=None):
    '''
        This function calculates the mean specific angular momentum for the given particle type, 
        for a given galaxy within the specified radius

        PARAMETERS:
        -----------
            filename: 'str'
                the name of the file containing galaxy information
            ptype: 'int' (i.e. 1, 2, or 3)
                the particle type (1 for halo, 2 for disk)
            r_max: 'float'
                the max radius of considered particles

        OUTPUT:
        -------
            L: np.array
                the normalized specific angular momentum vector 
                of the particles of a given type within r_max
    '''
    
    # galaxy center of mass computed from disk properties. 
    galaxy_COM = CenterOfMass(filename, 2)
    
    # storing the COM position and COM velocity of the disk
    r_COM = galaxy_COM.COM_P(0.1)
    v_COM = galaxy_COM.COM_V(r_COM[0], r_COM[1], r_COM[2])
    
    
    # creating a new instance of center of mass to load in the
    # particle positions and velocities of the desired type
    ptype_data = CenterOfMass(filename, ptype)
    
    # Following Lab 7 
    # Determine positions of the desired particles relative to COM 
    x = ptype_data.x - r_COM[0].value 
    y = ptype_data.y - r_COM[1].value 
    z = ptype_data.z - r_COM[2].value 

    # total magnitude
    rtot = np.sqrt(x**2 + y**2 + z**2)

    # Determine velocities of desired particles relative to COM motion
    vx = ptype_data.vx - v_COM[0].value 
    vy = ptype_data.vy - v_COM[1].value 
    vz = ptype_data.vz - v_COM[2].value 

    # Arrays for r and v 
    r = np.array([x,y,z]).T # transposed 
    v = np.array([vx,vy,vz]).T
    
    # identify the particles that are within the desired radius
    index = np.where(rtot < r_max)
    
    # From RotateFrame in Lab 7:
    # compute the specific angular momentum for all particles within the desired radius
    L = np.sum(np.cross(r[index[0],:],v[index[0],:]), axis=0)
    
    # normalize the specific angular momentum vector
    L_norm = L/np.sqrt(np.sum(L**2))

    return L_norm

## computing CosTheta between the MW Disk and Halo at Snap 0

In [72]:
# Mean specific angular momentum of the MW disk within 15 kpc at Snap 0
# Using the HIGH RES FILES

MW_DiskAng_Snap0 = AngularMomentum("MW_000_HighRes.txt", 2, 15)
print(MW_DiskAng_Snap0)
# disk angular momentum is in the -z direction, as expected. 
# MW Disk is rotating CW 


[-0.00463909 -0.00700256 -0.99996472]


In [73]:
# double checking that the MW ang momentum vector is normalized
print("magnitude", np.sqrt(MW_DiskAng_Snap0[0]**2 + MW_DiskAng_Snap0[1]**2 + MW_DiskAng_Snap0[2]**2))

magnitude 1.0


In [78]:
# Mean specific angular momentum of the MW halo within 20 kpc at Snap 0
MW_HaloAng_Snap0 = AngularMomentum("MW_000_HighRes.txt", 1, 20)
print(MW_HaloAng_Snap0)
# halo angular momentum is not perfectly aligned, but close enough

[-0.36415806  0.01643099 -0.93119221]


In [79]:
# Cos theta for MW halo and Disk (just the dot product since already normalized the vectors)

np.dot(MW_HaloAng_Snap0, MW_DiskAng_Snap0)

# so pretty aligned. 

0.9327336622916371

## computing CosTheta between the M31 Disk and Halo at Snap 0

In [70]:
# Mean specific angular momentum of the M31 disk within 15 kpc at Snap 0
M31_DiskAng_Snap0 = AngularMomentum("M31_000_HighRes.txt", 2, 15)
print(M31_DiskAng_Snap0)
# M31 disk is not aligned with z axis, as expected. 

[-0.41246108 -0.77011987 -0.48661201]


In [75]:
# Mean specific angular momentum of the M31 halo within 20 kpc at Snap 0
M31_HaloAng_Snap0 = AngularMomentum("M31_000_HighRes.txt", 1, 20)
print(M31_HaloAng_Snap0)
# halo ang mometum is not that far off the disk

[-0.53494497 -0.80396126 -0.25976947]


In [77]:
# Cos theta for M31 halo and Disk

np.dot(M31_HaloAng_Snap0, M31_DiskAng_Snap0)

# so M31 is starting pretty aligned

0.9661974596409755